<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_table('train.tsv', sep = '\t')
df_test = pd.read_table('test.tsv', sep = '\t', header = None)

In [ ]:
df_train

In [ ]:
df_train.shape

In [ ]:
plt.figure(figsize = (12,10))
sns.countplot(df_train.is_fake)

# ***Предобработка***

In [ ]:
import string

def remove_punctuation(text):
  return "".join([ch if ch not in string.punctuation else ' ' for ch in text])


def remove_numbers(text):
  return "".join([i if not i.isdigit() else ' ' for i in text])  


import re
def remove_multiple_spaces(text):  
  return re.sub(r'\s+', ' ', text, flags=re.I)

import nltk
from nltk.stem import *
nltk.download('all')
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

mystem = Mystem()

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['...', '"'])



def lemmatize_text(text):
  tokens = mystem.lemmatize(text.lower())
  tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
  text = " ".join(tokens)
  return text

In [ ]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_train['title'])]

In [ ]:
len(prep_text)

In [ ]:
prep_text[0]

In [ ]:
df_train['text_pred'] = prep_text

In [ ]:
df_train.head()

# ***Стемминг***

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

In [ ]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['т.д.', 'т', 'д'])

In [ ]:
from nltk import word_tokenize

stemmed_texts_list = []
for text in tqdm(df_train['text_pred']):
    tokens = word_tokenize(text)    
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_train['text_stem'] = stemmed_texts_list

In [ ]:
df_train.head()

In [ ]:
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [ ]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(df_train['text_pred']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df_train['text_sw'] = sw_texts_list

In [ ]:
df_train.head()

# *** Лемматизация***

In [ ]:
'''
lemm_texts_list = []
for text in tqdm(df_train['text_sw']):
    #print(text)
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df_train['text_lemm'] = lemm_texts_list

'''

In [ ]:
X = df_train['text_sw']
y = df_train['is_fake']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
my_tags = df_train['is_fake'].unique()
my_tags

# ***Naive Bayes Classifier***

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
%%time
nb.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [ ]:
%%time
logreg.fit(X_train, y_train)

In [ ]:
%%time
y_pred = logreg.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

Linear Support Vector Machine

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
%%time
sgd.fit(X_train, y_train)

In [ ]:
%%time
y_pred = sgd.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))